In [2]:
import pandas as pd

In [ ]:
def get_end_date(start_date: str) -> str:
    end_date = datetime.now().strftime('%Y-%m-%d')
    return end_date

In [ ]:
def get_num_cycles(start_date: str) -> int:
    # Get today's date
    today_date = datetime.now().strftime('%Y-%m-%d')
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
    end_datetime = datetime.strptime(today_date, '%Y-%m-%d')
    date_difference = (end_datetime - start_datetime).days

    return date_difference

In [ ]:
def train_test_split_for_data(data: pd.DataFrame, date_col: str, start_date: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train_data = data[data[date_col] <= start_date]
    test_data = data[(data[date_col] > start_date) & (data[date_col] <= get_end_date(start_date))]
    return train_data, test_data

In [11]:
user_interest_df = pd.read_feather('../../datasets/raw/user_interest.feather')
user_df = pd.read_feather('../../datasets/raw/user.feather')
video_df = pd.read_feather('../../datasets/raw/video.feather')
vote_df = pd.read_feather('../../datasets/raw/vote.feather')
season_df = pd.read_feather('../../datasets/raw/season.feather')

In [19]:
df = user_df.merge(user_interest_df, left_on="id", right_on="user_id", how="left")
df = df.merge(vote_df, left_on="id", right_on="voter_id", how="left", suffixes=["_user", "_video"])


In [22]:
df

,id_user,type,age_consent,created_at_x,user_id,name,recce_status,created_at_y,updated_at,id_video,video_id,voter_id,season_id,status,timestamp,video_creator_id,created_at
0,00009b00-b3ae-458a-841e-e3ad5f1a2b3c,PROFILE,0,2023-07-30 02:54:06,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,00009c76-592e-46a3-8754-59d68e1a2b3c,PROFILE,1,2023-06-15 02:48:48,NaN,NaN,NaN,NaT,NaT,a064b6c2-a4a1-4969-a34b-a3adf33b7360,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,00009c76-592e-46a3-8754-59d68e1a2b3c,43fda735-831b-41e7-9100-39657e1a2b3c,UPVOTE,2023-06-16T07:54:37.231Z,8f849ec4-b434-49d9-9b0d-c3e5171a2b3c,2023-06-16 07:55:20
2,00009c76-592e-46a3-8754-59d68e1a2b3c,PROFILE,1,2023-06-15 02:48:48,NaN,NaN,NaN,NaT,NaT,ac875152-2742-4dab-ad8d-531fbf71f11f,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,00009c76-592e-46a3-8754-59d68e1a2b3c,43fda735-831b-41e7-9100-39657e1a2b3c,UPVOTE,2023-06-20T13:07:30.207Z,8f849ec4-b434-49d9-9b0d-c3e5171a2b3c,2023-06-20 13:08:07
3,00009c76-592e-46a3-8754-59d68e1a2b3c,PROFILE,1,2023-06-15 02:48:48,NaN,NaN,NaN,NaT,NaT,18a4ae40-d219-4155-89d5-167f303cc702,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,00009c76-592e-46a3-8754-59d68e1a2b3c,43fda735-831b-41e7-9100-39657e1a2b3c,UPVOTE,2023-06-21T13:25:33.043Z,8f849ec4-b434-49d9-9b0d-c3e5171a2b3c,2023-06-21 13:26:15
4,00009c76-592e-46a3-8754-59d68e1a2b3c,PROFILE,1,2023-06-15 02:48:48,NaN,NaN,NaN,NaT,NaT,4577c846-c4e3-4eaa-98da-78b8b4b8cf2f,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,00009c76-592e-46a3-8754-59d68e1a2b3c,43fda735-831b-41e7-9100-39657e1a2b3c,UPVOTE,2023-06-23T08:48:00.545Z,8f849ec4-b434-49d9-9b0d-c3e5171a2b3c,2023-06-23 08:48:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212447,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,PROFILE,1,2023-05-14 02:51:25,NaN,NaN,NaN,NaT,NaT,3ce2318d-46d8-4495-a2c7-a12662859b9b,95c9270c-e6db-4520-823a-447aee1a2b3c,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,UPVOTE,2023-05-15T06:32:03.206Z,5d008417-eac0-402c-ba56-ecfaed1a2b3c,2023-05-15 06:32:29
212448,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,PROFILE,1,2023-05-14 02:51:25,NaN,NaN,NaN,NaT,NaT,c41f66b6-fa1e-4698-ad09-e77f8dd10305,c05f4ba5-8830-4be5-808c-8706e81a2b3c,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,UPVOTE,2023-05-14T02:41:57.733Z,db9f2b71-f024-41eb-b4c3-db54eb1a2b3c,2023-05-14 02:42:01
212449,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,PROFILE,1,2023-05-14 02:51:25,NaN,NaN,NaN,NaT,NaT,51a9e28f-6d64-4f14-b83e-28f1b40697b9,c05f4ba5-8830-4be5-808c-8706e81a2b3c,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,UPVOTE,2023-05-15T06:32:09.298Z,db9f2b71-f024-41eb-b4c3-db54eb1a2b3c,2023-05-15 06:32:57
212450,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,PROFILE,1,2023-05-14 02:51:25,NaN,NaN,NaN,NaT,NaT,3603a275-9519-47cc-bc16-9d3859df236d,da41a232-3db6-46b5-b51f-7823081a2b3c,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,UPVOTE,2023-05-14T02:41:59.494Z,d4402660-a592-4b10-aa1c-b75a6b1a2b3c,2023-05-14 02:42:31
